In [1]:
import spacy
import json
import random
import os
import ast
from google.colab import drive
from spacy.training import Example
from spacy.scorer import Scorer
from spacy.tokens import Doc
from spacy import displacy
from spacy.training import Example
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelEncoder
!python -m spacy download es_core_news_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.0/568.0 MB 3.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [4]:
drive.mount('/content/drive')
tfmPath = '/content/drive/MyDrive/TFM'
trainingFilesPath = tfmPath+'/archivos_entrenamiento'
validationFilePath = tfmPath+'/archivos_validacion'

Mounted at /content/drive


In [5]:
archivo = trainingFilesPath+'/train.txt'

with open(archivo, 'r', encoding='utf-8') as file:
    content = file.read()
    data = ast.literal_eval(content)


In [6]:
archivo_val = trainingFilesPath+'/val.txt'

with open(archivo, 'r', encoding='utf-8') as file:
    content = file.read()
    data_val = ast.literal_eval(content)


In [7]:
def entrenar(datos, iteraciones=10):
    datos_gold = datos
    nlp = spacy.blank('es')
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe("ner", last=True)
    for _, anns in datos_gold:
        for ent in anns.get('entities'):
            ner.add_label(ent[2])

    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.select_pipes(disable=other_pipes):
        optimizer = nlp.begin_training()
        for n in range(iteraciones):
            print("Iteración número " + str(n))
            random.shuffle(datos_gold)
            losses = {}
            for texto, anns in datos_gold:
              doc = nlp.make_doc(texto)
              ejemplo = Example.from_dict(doc, anns)
              nlp.update(
                  [ejemplo],
                  drop=0.2,
                  sgd=optimizer,
                  losses=losses)
            print(losses)
    return nlp

In [8]:
entrenamiento = entrenar(data)

Iteración número 0
{'ner': 1837.8907753982476}
Iteración número 1
{'ner': 923.1260107646568}
Iteración número 2
{'ner': 770.3539020354212}
Iteración número 3
{'ner': 559.9162439070041}
Iteración número 4
{'ner': 504.71138200796173}
Iteración número 5
{'ner': 404.96100695851965}
Iteración número 6
{'ner': 363.3142258403718}
Iteración número 7
{'ner': 312.2590729290203}
Iteración número 8
{'ner': 301.15295898265094}
Iteración número 9
{'ner': 252.33800909852977}


In [9]:
path_modelo = tfmPath+'/modelo'
entrenamiento.to_disk(path_modelo)

In [10]:
entrenamiento.to_disk("modelo")

In [11]:
nlp = spacy.load(path_modelo)

In [ ]:

def my_evaluate(ner_model, examples):
    true_labels = []
    pred_labels = []

    for input_, annotations in examples:
        doc = ner_model(input_)

        true_entities = ["O"] * len(input_)
        for start, end, label in annotations['entities']:
            true_entities[start:end] = [label] * (end - start)

        pred_entities = ["O"] * len(input_)
        for ent in doc.ents:
            pred_entities[ent.start_char:ent.end_char] = [ent.label_] * (ent.end_char - ent.start_char)

        true_labels.extend(true_entities)
        pred_labels.extend(pred_entities)

    precision = precision_score(true_labels, pred_labels, average='weighted', zero_division=1)
    recall = recall_score(true_labels, pred_labels, average='weighted', zero_division=1)
    f1 = f1_score(true_labels, pred_labels, average='weighted', zero_division=1)

    return {
        'precision': precision,
        'recall': recall,
        'f1_score': f1
    }

results = my_evaluate(nlp, data_val)
print(f"Precisión: {results['precision']}")
print(f"Cobertura (Recall): {results['recall']}")
print(f"F1-score: {results['f1_score']}")


Precisión: 0.9975262019528743
Cobertura (Recall): 0.9973815247252747
F1-score: 0.9973631839333628


In [ ]:
def my_evaluate(ner_model, examples):
    true_labels = []
    pred_labels = []
    all_entity_labels = set()

    for input_, annotations in examples:
        doc = ner_model(input_)

        true_entities = ["O"] * len(input_)
        for start, end, label in annotations['entities']:
            true_entities[start:end] = [label] * (end - start)
            all_entity_labels.add(label)

        pred_entities = ["O"] * len(input_)
        for ent in doc.ents:
            pred_entities[ent.start_char:ent.end_char] = [ent.label_] * (ent.end_char - ent.start_char)
            all_entity_labels.add(ent.label_)


        true_labels.extend(true_entities)
        pred_labels.extend(pred_entities)

    label_encoder = LabelEncoder()
    all_entity_labels = list(all_entity_labels)
    label_encoder.fit(all_entity_labels + ["O"])

    true_labels_encoded = label_encoder.transform(true_labels)
    pred_labels_encoded = label_encoder.transform(pred_labels)

    precision_per_entity = precision_score(true_labels_encoded, pred_labels_encoded, labels=label_encoder.transform(all_entity_labels), average=None, zero_division=1)
    recall_per_entity = recall_score(true_labels_encoded, pred_labels_encoded, labels=label_encoder.transform(all_entity_labels), average=None, zero_division=1)
    f1_per_entity = f1_score(true_labels_encoded, pred_labels_encoded, labels=label_encoder.transform(all_entity_labels), average=None, zero_division=1)

    results_per_entity = {}
    for i, label in enumerate(all_entity_labels):
        results_per_entity[label] = {
            'precision': precision_per_entity[i],
            'recall': recall_per_entity[i],
            'f1_score': f1_per_entity[i]
        }

    return results_per_entity

results = my_evaluate(nlp, data_val)

for entity, metrics in results.items():
    print(f"Entidad: {entity}")
    print(f"  Precisión: {metrics['precision']}")
    print(f"  Cobertura (Recall): {metrics['recall']}")
    print(f"  F1-score: {metrics['f1_score']}")


Entidad: dis3
  Precisión: 1.0
  Cobertura (Recall): 0.968789013732834
  F1-score: 0.984147114774889
Entidad: dis5
  Precisión: 0.985734664764622
  Cobertura (Recall): 1.0
  F1-score: 0.9928160919540231
Entidad: vdis1
  Precisión: 1.0
  Cobertura (Recall): 1.0
  F1-score: 1.0
Entidad: dis4
  Precisión: 0.9864197530864197
  Cobertura (Recall): 1.0
  F1-score: 0.9931634555624612
Entidad: hab
  Precisión: 0.9532710280373832
  Cobertura (Recall): 1.0
  F1-score: 0.9760765550239235
Entidad: vdis2
  Precisión: 1.0
  Cobertura (Recall): 1.0
  F1-score: 1.0
Entidad: inv
  Precisión: 0.9906542056074766
  Cobertura (Recall): 0.9906542056074766
  F1-score: 0.9906542056074766
Entidad: dis2
  Precisión: 0.9844760672703752
  Cobertura (Recall): 0.7145539906103286
  F1-score: 0.8280739934711644
Entidad: ing
  Precisión: 0.9369369369369369
  Cobertura (Recall): 1.0
  F1-score: 0.9674418604651163
Entidad: cl
  Precisión: 0.8852459016393442
  Cobertura (Recall): 1.0
  F1-score: 0.9391304347826086
Entida

In [ ]:
texto_input = input("Por favor, introduce el texto: ")


Por favor, introduce el texto:En el ámbito de la atención sanitaria, es fundamental brindar apoyo adecuado a quienes enfrentan diversas condiciones de salud. Por ejemplo, el discapacitado puede requerir acceso a terapias de rehabilitación especializadas, mientras que el minusválido a menudo necesita adaptaciones en su entorno para mejorar su movilidad y calidad de vida. De manera similar, el inválido puede beneficiarse de dispositivos de asistencia tecnológica que le permitan realizar tareas cotidianas de forma más independiente.  Existen condiciones más específicas, como la de un tetrapléjico, que generalmente requiere cuidados médicos continuos y soporte especializado para manejar su situación. Además, quienes padecen discapacidad a veces deben enfrentar barreras en el acceso a la educación y el empleo, lo cual limita sus oportunidades de desarrollo personal y profesional.  Por otro lado, algunos individuos que sufren discapacidad debido a accidentes o enfermedades pueden experimenta

In [ ]:
doc = nlp(texto_input)
displacy.render(doc, style='ent', jupyter=True)

In [ ]:
if len(doc.ents) == 0:
  print(f"No se han encontrado mejoras según los criterios establecidos.")

for ent in doc.ents:
    if ent.label_ == "alu":
        print(f"Considere cambiar {ent.text} por 'el alumnado'. ")
    if ent.label_ == "profs":
        print(f"Considere cambiar {ent.text} por 'el profesorado'. ")
    if ent.label_ == "inv":
        print(f"Considere cambiar {ent.text} por 'el equipo de investigación'. ")
    if ent.label_ == "hab":
      print(f"Considere cambiar {ent.text} por 'la población'. ")
    if ent.label_ == "cam":
      print(f"Considere cambiar {ent.text} por 'el personal de servicio'. ")
    if ent.label_ == "ing":
      print(f"Considere cambiar {ent.text} por 'el equipo de ingeniería'. ")
    if ent.label_ == "bus":
      print(f"Considere cambiar {ent.text} por 'las personas de negocios'. ")
    if ent.label_ == "cl":
      print(f"Considere cambiar {ent.text} por 'la clientela' ")
    if ent.label_ == "dis1" or ent.label_ == "dis2" or ent.label_ == "dis3" or ent.label_ == "dis4" or ent.label_ == "dis5":
      print(f"Considere cambiar {ent.text} por '(persona) con discapacidad'.")
    if ent.label_ == "dis6":
      print(f"Considere cambiar {ent.text} por 'la persona con tetrapléjia'.")
    if ent.label_ == "vdis1" or ent.label_ == "vdis2":
      print(f"Considere cambiar {ent.text} por 'tener discapacidad' ")

Considere cambiar discapacitado por 'la persona con discapacidad'.
Considere cambiar minusválido por 'la persona con discapacidad'.
Considere cambiar inválido por 'la persona con discapacidad'.
Considere cambiar tetrapléjico por 'la persona con tetrapléjia'.
Considere cambiar padecen discapacidad por 'tener discapacidad' 
Considere cambiar sufren discapacidad por 'tener discapacidad' 
Considere cambiar retrasado por 'la persona con discapacidad'.
